## Set Up Environment and Import Data

In [18]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from scipy import stats

sns.set_theme()

In [2]:
FILENAME='Case Study Data - Comp Research Associate(1)'
SHEET1 = 'recruiting_activity'
SHEET2 = 'candidate_profile'

In [3]:
# Import datasets separately
df_rec = pd.read_excel("{}.xlsx".format(FILENAME), sheet_name=SHEET1)
df_prof = pd.read_excel("{}.xlsx".format(FILENAME), sheet_name=SHEET2)

## Profiling

In [4]:
# Get dimensions of datasets
print(df_rec.shape)
print(df_prof.shape)

(4996, 6)
(4959, 10)


In [ ]:
# This confirms that we have duplicate candidate id in the df_rec dataframe
df_rec.cand_id.unique().shape

(4959,)

In [ ]:
# This confirms that we do not have duplicate candidate id in the df_prof dataframe
df_prof.cand_id.unique().shape

(4959,)

In [5]:
# This confirms that there is a perfect inetersect of employees from one dataset to the other
print([x for x in df_rec.cand_id.unique() if x not in df_prof.cand_id.unique()])
print([x for x in df_prof.cand_id.unique() if x not in df_rec.cand_id.unique()])

[]
[]


In [6]:
df = df_rec.merge(df_prof, on='cand_id', how='left')
df.head()

,cand_id,position,dept,furthest_stage,app_date,app_source,comp_exp,yrs_exp,industry,degree_1,school_region_1,degree_2,school_region_2,degree_3,school_region_3
0,4188467,UX Designer,Product,New Application,2018-02-03 00:00:00,Agency,75000,14.000000,Technology,Bachelors,region_5,Masters,region_5,NaN,NaN
1,433034,UX Designer,Product,In-House Interview,2018-12-20 00:00:00,Website,74900,13.951020,Other,Bachelors,region_4,Masters,region_1,NaN,NaN
2,337226,UX Designer,Product,In-House Interview,2017-02-27 00:00:00,Agency,73400,13.216327,Other,Bachelors,region_4,Masters,region_4,NaN,NaN
3,6108441,UX Designer,Product,Phone Screen,2017-06-15 00:00:00,Agency,72200,12.628571,Technology,Bachelors,region_5,NaN,NaN,NaN,NaN
4,1536927,UX Designer,Product,In-House Interview,2017-10-18 00:00:00,Advertisement,71900,12.481633,Technology,Bachelors,region_4,Masters,region_4,NaN,NaN


In [7]:
df.describe()

,cand_id,comp_exp,yrs_exp
count,4.996000e+03,4996.000000,4996.000000
mean,2.450057e+06,72015.632506,17.816077
std,2.303560e+06,11504.386499,9.579444
min,2.468000e+03,1500.000000,0.000000
25%,2.398512e+05,64400.000000,9.044776
50%,1.662334e+06,70900.000000,14.820896
75%,4.605206e+06,79200.000000,27.614754
max,6.377786e+06,450000.000000,36.262295


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4996 entries, 0 to 4995
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   cand_id          4996 non-null   int64  
 1   position         4996 non-null   object 
 2   dept             4996 non-null   object 
 3   furthest_stage   4996 non-null   object 
 4   app_date         4996 non-null   object 
 5   app_source       4996 non-null   object 
 6   comp_exp         4996 non-null   int64  
 7   yrs_exp          4996 non-null   float64
 8   industry         4996 non-null   object 
 9   degree_1         4996 non-null   object 
 10  school_region_1  4996 non-null   object 
 11  degree_2         2142 non-null   object 
 12  school_region_2  2142 non-null   object 
 13  degree_3         914 non-null    object 
 14  school_region_3  914 non-null    object 
dtypes: float64(1), int64(2), object(12)
memory usage: 624.5+ KB


In [10]:
df.isna().sum()

cand_id               0
position              0
dept                  0
furthest_stage        0
app_date              0
app_source            0
comp_exp              0
yrs_exp               0
industry              0
degree_1              0
school_region_1       0
degree_2           2854
school_region_2    2854
degree_3           4082
school_region_3    4082
dtype: int64

In [11]:
df.groupby(['industry','dept','position']).size()

industry    dept         position                      
Finance     Engineering  Associate Software Developer      542
                         Sr. Software Engineer              10
            Finance      Business Operations Manager        34
                         Finance Generalist                 30
                         Finance Manager                    30
                         Financial Analyst                   9
            IT           IT Analyst                         16
                         Sr. Business Analyst               31
            Product      Associate Product Manager         151
                         Sr. Product Manager                17
                         UX Designer                        11
            Sales        Account Executive                  36
                         Associate Relationship Manager    345
Government  Engineering  Associate Software Developer      500
                         Sr. Software Engineer               9

In [17]:
# Use for markdown
print(df.head().to_markdown())

|      |   cand_id | position                       | dept        | furthest_stage   | app_date            | app_source       |   comp_exp |   yrs_exp | industry   | degree_1   | school_region_1   | degree_2   | school_region_2   | degree_3   | school_region_3   | Highest Degree   |
|-----:|----------:|:-------------------------------|:------------|:-----------------|:--------------------|:-----------------|-----------:|----------:|:-----------|:-----------|:------------------|:-----------|:------------------|:-----------|:------------------|:-----------------|
|  820 |      2468 | Associate Software Developer   | Engineering | Offer Sent       | 2018-11-01 00:00:00 | Campus Job Board |      85100 |  36.2623  | Government | PhD        | region_5          | Masters    | region_4          | Bachelors  | region_2          | PhD              |
| 4002 |      2471 | Associate Relationship Manager | Sales       | Offer Sent       | 2018-12-13 00:00:00 | Campus Event     |      74700 |   8.373

## Adding Highest Degree

In [12]:
df['Degree Concat'] = df.degree_1.astype(str)+df.degree_2.astype(str)+df.degree_3.astype(str)

df['Highest Degree'] = np.where(df['Degree Concat'].str.contains('PhD'), "PhD",
                                np.where(df['Degree Concat'].str.contains('JD'), "JD", 
                                         np.where(df['Degree Concat'].str.contains('Masters'), "Masters", 
                                                  np.where(df['Degree Concat'].str.contains('Bachelors'), "Bachelors", "")
                                                  )
                                         )
                                )

df = df.drop(['Degree Concat'], axis=1)

In [13]:
df['Highest Degree'] .value_counts()

Bachelors    2854
Masters      1228
PhD           914
Name: Highest Degree, dtype: int64

## Deduplications

In [14]:
# Creating a numerical hierarchy for the stages reached
# Ordering candidates based on this hierarchy so that duplication retains only the application with the latest stage
df['furthers_stage_enc'] = df["furthest_stage"].replace({
    'New Application':1,
    'Phone Screen':2,
    'In-House Interview':3,
    'Offer Sent':4
})
df = df.sort_values(['cand_id', 'dept', 'furthest_stage']).drop_duplicates(['cand_id', 'dept'], keep='last')
df = df.drop(['furthers_stage_enc'], axis=1)

## Funnel

In [15]:
# Creating funnel by pivoting our data accordingly
funnel = df.pivot_table(values='cand_id', index=['furthest_stage'], columns='Highest Degree', aggfunc='count')
funnel = funnel.reindex(['New Application', 'Phone Screen','In-House Interview',  'Offer Sent'])
funnel['Bachelors Conversion Rate'] = funnel['Bachelors'].transform(lambda x: x / x.shift(1))

funnel['Masters Conversion Rate'] = funnel['Masters'].transform(lambda x: x / x.shift(1))
funnel['PhD Conversion Rate'] = funnel['PhD'].transform(lambda x: x /x.shift(1))

funnel = funnel.rename({'Bachelors':'Bachelors Applicants',
               'Masters':'Masters Applicants',
               'PhD':'PhD Applicants'}, axis=1)

funnel = funnel.reindex(sorted(funnel.columns), axis=1)
funnel = funnel.fillna(0)

funnel.style.format({
    'Bachelors Conversion Rate': '{:,.0%}'.format,
    'Masters Conversion Rate': '{:,.0%}'.format,
    'PhD Conversion Rate': '{:,.0%}'.format,
})

Highest Degree,Bachelors Applicants,Bachelors Conversion Rate,Masters Applicants,Masters Conversion Rate,PhD Applicants,PhD Conversion Rate
furthest_stage,,,,,,
New Application,2182,0%,842,0%,354,0%
Phone Screen,342,16%,174,21%,264,75%
In-House Interview,250,73%,172,99%,262,99%
Offer Sent,64,26%,30,17%,30,11%


In [16]:
# Use for markdown
print(funnel.to_markdown())

| furthest_stage     |   Bachelors Applicants |   Bachelors Conversion Rate |   Masters Applicants |   Masters Conversion Rate |   PhD Applicants |   PhD Conversion Rate |
|:-------------------|-----------------------:|----------------------------:|---------------------:|--------------------------:|-----------------:|----------------------:|
| New Application    |                   2182 |                    0        |                  842 |                  0        |              354 |              0        |
| Phone Screen       |                    342 |                    0.156737 |                  174 |                  0.206651 |              264 |              0.745763 |
| In-House Interview |                    250 |                    0.730994 |                  172 |                  0.988506 |              262 |              0.992424 |
| Offer Sent         |                     64 |                    0.256    |                   30 |                  0.174419 |            

## Regression

### Overall Regression

In [19]:
# Run regression for overall data using only subset of columns
cols = [
    # 'position', 
        'dept',  'furthest_stage', 'app_source', 'comp_exp', 'yrs_exp', 'industry','Highest Degree']

X = pd.get_dummies(df[cols], drop_first='True').drop(['comp_exp'], axis=1)
y = np.log(df.comp_exp)

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:               comp_exp   R-squared:                       0.743
Model:                            OLS   Adj. R-squared:                  0.742
Method:                 Least Squares   F-statistic:                     713.4
Date:                Wed, 05 Apr 2023   Prob (F-statistic):               0.00
Time:                        20:00:12   Log-Likelihood:                 5740.6
No. Observations:                4966   AIC:                        -1.144e+04
Df Residuals:                    4945   BIC:                        -1.130e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const       

In [29]:
def format_coef(est):
    df = pd.DataFrame.from_records(est.summary().tables[1].data)
    header  = df.iloc[0] # grab the first row for the header
    df = df[1:] # take the data less the header row
    df.columns = header 
    df['Exp. Coef.'] = np.exp(df['coef'].astype(float))-1
    return df

In [31]:
coef_ovr = format_coef(est2)
coef_ovr

,,coef,std err,t,P>|t|,[0.025,0.975],Exp. Coef.
1,const,10.3581,0.011,950.260,0.000,10.337,10.379,31510.253028
2,yrs_exp,0.0256,0.000,74.421,0.000,0.025,0.026,0.025930
3,dept_Finance,0.5256,0.007,76.319,0.000,0.512,0.539,0.691473
4,dept_IT,0.4765,0.009,55.540,0.000,0.460,0.493,0.610428
5,dept_Product,0.5704,0.008,74.001,0.000,0.555,0.586,0.768974
6,dept_Sales,0.6530,0.007,97.057,0.000,0.640,0.666,0.921296
7,furthest_stage_New Application,0.0068,0.003,2.017,0.044,0.000,0.013,0.006823
8,furthest_stage_Offer Sent,-0.0014,0.007,-0.188,0.851,-0.016,0.013,-0.001399
9,furthest_stage_Phone Screen,-0.0017,0.004,-0.434,0.664,-0.010,0.006,-0.001699
10,app_source_Agency,0.0735,0.006,11.875,0.000,0.061,0.086,0.076269


In [20]:
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)  for i in range(len(X.columns))]

vif_data

,feature,VIF
0,yrs_exp,9.630225
1,dept_Finance,2.198069
2,dept_IT,1.243027
3,dept_Product,2.233854
4,dept_Sales,3.243485
5,furthest_stage_New Application,5.909276
6,furthest_stage_Offer Sent,1.176481
7,furthest_stage_Phone Screen,2.069994
8,app_source_Agency,1.770793
9,app_source_Campus Event,2.250839


### Regression by Department

In [33]:
def regress(df):
    cols2 = [ 'position', 'furthest_stage', 'app_source', 'comp_exp', 'yrs_exp', 'industry','Highest Degree']

    X = pd.get_dummies(df[cols2], drop_first='True').drop(['comp_exp'], axis=1)
    y = np.log(df.comp_exp)

    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est = est.fit()
    return est

In [37]:
regression_tracker = {}
for dept in df.dept.unique():
    est_temp = regress(df.loc[df.dept==dept])
    regression_tracker.update({dept:est_temp})
    print("------------ {} --------------".format(dept))
    print(est_temp.summary().tables[0])

------------ Engineering --------------
                            OLS Regression Results                            
Dep. Variable:               comp_exp   R-squared:                       0.601
Model:                            OLS   Adj. R-squared:                  0.598
Method:                 Least Squares   F-statistic:                     198.1
Date:                Wed, 05 Apr 2023   Prob (F-statistic):               0.00
Time:                        20:08:27   Log-Likelihood:                 2217.9
No. Observations:                2120   AIC:                            -4402.
Df Residuals:                    2103   BIC:                            -4306.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
------------ Sales --------------
                            OLS Regression Results                            
Dep. Variable:               comp_exp   R-squared:       

#### Finance

In [38]:
## Finance
coef_fin = format_coef(regression_tracker['Finance'])
coef_fin

,,coef,std err,t,P>|t|,[0.025,0.975],Exp. Coef.
1,const,10.9430,0.007,1495.766,0.000,10.929,10.957,56555.759173
2,yrs_exp,0.0222,0.000,67.664,0.000,0.022,0.023,0.022448
3,position_Finance Generalist,-0.0850,0.008,-11.147,0.000,-0.100,-0.070,-0.081488
4,position_Finance Manager,0.1104,0.003,41.097,0.000,0.105,0.116,0.116725
5,position_Financial Analyst,-0.1458,0.007,-21.147,0.000,-0.159,-0.132,-0.135669
6,furthest_stage_New Application,0.0038,0.003,1.311,0.191,-0.002,0.010,0.003807
7,furthest_stage_Offer Sent,0.0046,0.006,0.829,0.407,-0.006,0.016,0.004611
8,furthest_stage_Phone Screen,0.0051,0.003,1.478,0.140,-0.002,0.012,0.005113
9,app_source_Agency,0.0014,0.004,0.334,0.739,-0.007,0.010,0.001401
10,app_source_Campus Event,0.0004,0.010,0.038,0.969,-0.019,0.019,0.000400


#### IT

In [39]:
## IT
coef_it = format_coef(regression_tracker['IT'])
coef_it

,,coef,std err,t,P>|t|,[0.025,0.975],Exp. Coef.
1,const,10.8562,0.009,1156.190,0.000,10.838,10.875,51853.655631
2,yrs_exp,0.0183,0.001,35.923,0.000,0.017,0.019,0.018468
3,position_Sr. Business Analyst,0.1852,0.007,26.060,0.000,0.171,0.199,0.203459
4,furthest_stage_New Application,0.0060,0.006,0.991,0.323,-0.006,0.018,0.006018
5,furthest_stage_Offer Sent,0.0198,0.009,2.082,0.039,0.001,0.039,0.019997
6,furthest_stage_Phone Screen,0.0076,0.008,0.989,0.324,-0.008,0.023,0.007629
7,app_source_Agency,-0.0100,0.007,-1.405,0.162,-0.024,0.004,-0.009950
8,app_source_Campus Event,-0.0272,0.010,-2.618,0.010,-0.048,-0.007,-0.026833
9,app_source_Campus Job Board,-0.0160,0.009,-1.851,0.066,-0.033,0.001,-0.015873
10,app_source_Career Fair,-0.0242,0.008,-3.165,0.002,-0.039,-0.009,-0.023910


#### Product

In [40]:
coef_pr = format_coef(regression_tracker['Product'])
coef_pr

,,coef,std err,t,P>|t|,[0.025,0.975],Exp. Coef.
1,const,10.8855,0.004,3031.682,0.000,10.878,10.893,53395.474384
2,yrs_exp,0.0360,0.000,119.500,0.000,0.035,0.037,0.036656
3,position_Sr. Product Manager,-0.1841,0.004,-43.212,0.000,-0.192,-0.176,-0.168147
4,position_UX Designer,-0.1373,0.003,-50.310,0.000,-0.143,-0.132,-0.128291
5,furthest_stage_New Application,0.0020,0.002,1.174,0.241,-0.001,0.005,0.002002
6,furthest_stage_Offer Sent,0.0017,0.004,0.430,0.667,-0.006,0.009,0.001701
7,furthest_stage_Phone Screen,-0.0013,0.002,-0.672,0.502,-0.005,0.003,-0.001299
8,app_source_Agency,-0.0080,0.003,-2.360,0.019,-0.015,-0.001,-0.007968
9,app_source_Campus Event,0.0043,0.003,1.492,0.136,-0.001,0.010,0.004309
10,app_source_Campus Job Board,0.0045,0.002,1.901,0.058,-0.000,0.009,0.004510


#### Sales

In [41]:
coef_s = format_coef(regression_tracker['Sales'])
coef_s

,,coef,std err,t,P>|t|,[0.025,0.975],Exp. Coef.
1,const,11.0678,0.001,8106.951,0.000,11.065,11.071,64073.388008
2,yrs_exp,0.0275,8.5e-05,323.937,0.000,0.027,0.028,0.027882
3,position_Associate Relationship Manager,-0.0740,0.001,-79.782,0.000,-0.076,-0.072,-0.071328
4,furthest_stage_New Application,-0.0005,0.001,-0.825,0.410,-0.002,0.001,-0.000500
5,furthest_stage_Offer Sent,-0.0013,0.002,-0.750,0.453,-0.005,0.002,-0.001299
6,furthest_stage_Phone Screen,-0.0008,0.001,-0.978,0.328,-0.002,0.001,-0.000800
7,app_source_Agency,0.0075,0.002,4.829,0.000,0.004,0.011,0.007528
8,app_source_Campus Event,-0.0025,0.001,-2.919,0.004,-0.004,-0.001,-0.002497
9,app_source_Campus Job Board,-0.0026,0.001,-3.460,0.001,-0.004,-0.001,-0.002597
10,app_source_Career Fair,-0.0016,0.001,-1.748,0.081,-0.003,0.000,-0.001599


#### Engineering

In [42]:
coef_e = format_coef(regression_tracker['Engineering'])
coef_e

,,coef,std err,t,P>|t|,[0.025,0.975],Exp. Coef.
1,const,10.0714,0.024,418.768,0.000,10.024,10.119,23655.660922
2,yrs_exp,0.0357,0.001,42.900,0.000,0.034,0.037,0.036345
3,position_Sr. Software Engineer,0.5540,0.025,22.329,0.000,0.505,0.603,0.740200
4,furthest_stage_New Application,0.0012,0.006,0.185,0.854,-0.011,0.014,0.001201
5,furthest_stage_Offer Sent,-0.0077,0.013,-0.577,0.564,-0.034,0.018,-0.007670
6,furthest_stage_Phone Screen,-0.0014,0.007,-0.197,0.844,-0.016,0.013,-0.001399
7,app_source_Agency,0.0661,0.029,2.242,0.025,0.008,0.124,0.068334
8,app_source_Campus Event,0.0070,0.008,0.879,0.380,-0.009,0.023,0.007025
9,app_source_Campus Job Board,0.0056,0.007,0.851,0.395,-0.007,0.019,0.005616
10,app_source_Career Fair,0.0073,0.006,1.130,0.259,-0.005,0.020,0.007327


## Statistical Tests

In [45]:
df['PhD/xPhD'] = np.where(df['Highest Degree']=='PhD',"PhD", "Not PhD")

In [43]:
def expectation_differences(df, var, alpha=0.05):
    if var is None:
        A = df[(df['PhD/xPhD']=='PhD')]['comp_exp'].values.tolist()
        B = df[(df['PhD/xPhD']!='PhD')]['comp_exp'].values.tolist()
        t_check=stats.ttest_ind(A,B)
        sig = "Yes" if t_check[1]<alpha else "No"
        print("PhD Expectations: {:.0f}| XPhD Expectations: {:.0f} | Diff: {:.1%} | Sig?: {}".format(np.mean(A), np.mean(B), np.mean(A)/np.mean(B)-1, sig))
    else:
        for v in df[var].unique():
            A = df[(df['PhD/xPhD']=='PhD')&(df[var]==v)]['comp_exp'].values.tolist()
            B = df[(df['PhD/xPhD']!='PhD')&(df[var]==v)]['comp_exp'].values.tolist()
            t_check=stats.ttest_ind(A,B)
            sig = "Yes" if t_check[1]<alpha else "No"
            print(var, ": {} | PhD Expectations: {:.0f}| XPhD Expectations: {:.0f} | Diff: {:.1%} | Sig?: {}".format(v,np.mean(A), np.mean(B), np.mean(A)/np.mean(B)-1, sig))

In [46]:
expectation_differences(df, var=None)

PhD Expectations: 72440| XPhD Expectations: 71821 | Diff: 0.9% | Sig?: No


In [47]:
expectation_differences(df, var='dept')

dept : Engineering | PhD Expectations: 72213| XPhD Expectations: 63664 | Diff: 13.4% | Sig?: Yes
dept : Sales | PhD Expectations: 78435| XPhD Expectations: 79405 | Diff: -1.2% | Sig?: No
dept : Product | PhD Expectations: 70934| XPhD Expectations: 69797 | Diff: 1.6% | Sig?: No
dept : Finance | PhD Expectations: 77026| XPhD Expectations: 76781 | Diff: 0.3% | Sig?: No
dept : IT | PhD Expectations: 64906| XPhD Expectations: 75406 | Diff: -13.9% | Sig?: Yes


In [48]:
expectation_differences(df, var='industry')

industry : Government | PhD Expectations: 72520| XPhD Expectations: 71872 | Diff: 0.9% | Sig?: No
industry : Other | PhD Expectations: 73821| XPhD Expectations: 71844 | Diff: 2.8% | Sig?: No
industry : Finance | PhD Expectations: 71740| XPhD Expectations: 71766 | Diff: -0.0% | Sig?: No
industry : Technology | PhD Expectations: 71679| XPhD Expectations: 71804 | Diff: -0.2% | Sig?: No


In [49]:
expectation_differences(df, var='position')

position : Associate Software Developer | PhD Expectations: 70753| XPhD Expectations: 63538 | Diff: 11.4% | Sig?: Yes
position : Associate Relationship Manager | PhD Expectations: 79282| XPhD Expectations: 79337 | Diff: -0.1% | Sig?: No
position : Associate Product Manager | PhD Expectations: 70463| XPhD Expectations: 67841 | Diff: 3.9% | Sig?: Yes
position : Finance Manager | PhD Expectations: 91082| XPhD Expectations: 89238 | Diff: 2.1% | Sig?: No
position : Finance Generalist | PhD Expectations: 59760| XPhD Expectations: 57680 | Diff: 3.6% | Sig?: No
position : Business Operations Manager | PhD Expectations: 84108| XPhD Expectations: 83996 | Diff: 0.1% | Sig?: No
position : Sr. Business Analyst | PhD Expectations: 82733| XPhD Expectations: 82303 | Diff: 0.5% | Sig?: No
position : Sr. Product Manager | PhD Expectations: 87240| XPhD Expectations: 85291 | Diff: 2.3% | Sig?: No
position : IT Analyst | PhD Expectations: 58488| XPhD Expectations: 59000 | Diff: -0.9% | Sig?: No
position : 

In [60]:
agg = df.groupby(['PhD/xPhD','position']).agg({'cand_id':'count','comp_exp':'mean'}).reset_index().merge(df['PhD/xPhD'].value_counts().reset_index(), left_on=['PhD/xPhD'], right_on=['index'])
agg['Ratio'] = agg['cand_id']/agg['PhD/xPhD_y']
agg

,PhD/xPhD_x,position,cand_id,comp_exp,index,PhD/xPhD_y,Ratio
0,Not PhD,Account Executive,121,80134.710744,Not PhD,4056,0.029832
1,Not PhD,Associate Product Manager,443,67840.632054,Not PhD,4056,0.109221
2,Not PhD,Associate Relationship Manager,1302,79336.789555,Not PhD,4056,0.321006
3,Not PhD,Associate Software Developer,1493,63538.245144,Not PhD,4056,0.368097
4,Not PhD,Business Operations Manager,147,83995.918367,Not PhD,4056,0.036243
5,Not PhD,Finance Generalist,123,57680.487805,Not PhD,4056,0.030325
6,Not PhD,Finance Manager,145,89237.931034,Not PhD,4056,0.035750
7,Not PhD,Financial Analyst,25,56080.000000,Not PhD,4056,0.006164
8,Not PhD,IT Analyst,37,59000.000000,Not PhD,4056,0.009122
9,Not PhD,Sr. Business Analyst,88,82303.409091,Not PhD,4056,0.021696
